In [1]:
import spacy

print(spacy.__version__)

c:\Users\PC\anaconda3\envs\cv_parser\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.3.0


# Read and prepare data

In [2]:
import json
import os

CLASSES = ["PERSON_NAME","ADDRESS","EDUCATION","GPA","SKILL","EXPERIENCE_LEVEL","JOB_TITLE","DATE_BIRTH","MAJOR","MARIAGE_STATUS", 'GENDER', 'ORGANIZATION']



In [7]:
from pyvi import ViUtils
import re

doc_container = spacy.tokens.DocBin()
data_folder = '../data/resume_annotations'
nlp = spacy.blank('en')
WHITE_SPACE = re.compile(r'\s+')

len_txt = []
#Loop through list file in annotations folder
for file_name in (os.listdir(data_folder)):
    file_path = f'{data_folder}/{file_name}'
    
    if os.path.isfile(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

            if len(data['annotations']) == 0:
                continue
            
            annotations = data['annotations'][0]
            text = annotations[0]
            doc = nlp(text)
            len_txt.append(len(text))
            
            entities = annotations[1]['entities']
            
            ents_list = []
            for start_idx, end_idx, label in entities:
                span = doc.char_span(start_idx, end_idx, label = label, alignment_mode='contract')
                
                if WHITE_SPACE.match(str(span).strip()) or len(str(span)) == 0:
                    print("skip")
                    continue
                
                if span and label in CLASSES:
                    ents_list.append(span)
                    
                    
            ent_list = spacy.util.filter_spans(ents_list)
            doc.ents = ents_list

        doc_container.add(doc)
        

                

In [8]:
import numpy as np
np.median(len_txt)

139.0

In [9]:
import random
docs = list(doc_container.get_docs(nlp.vocab))

random.shuffle(docs)
for d in docs[:25]:
    print(d.ents)

(Vo Van Thinh, Senior Developer, Information Technology, Hue University, POS LACVIET, Senior Developer, POS system, POS system, research new technology, support team)
(C#, ASP.NET, MVC, ENTITY FRAMEWORK, JavaScript, CSS3, HTML5, LinQ, Angularjs, Kendo UI, VTA Vien Thong, Senior Developer, reports, analyzing, exploit data system, research new technology, support team member)
(POSTS AND TELECOMMUNICATIONS INSTITUTE, HD WEBSOFT, QC LEADER, Tester, Business Analyst, test plans, test cases, communications, automation testing, API testing, SOLUTION Tester, Business Analyst, Read requirement documents, Draw mockup, Write test cases, test plans, Report, track bugs, Manual testing, Report and track bugs, SCHOOL BLOOD DONATION ORGANIZATION)
(work in a team,)
(TRAN HOANG TRUONG HAI, Male, 8th February 1990, 32 block 1A Trung Nu Vuong street Cai Be Township Cai Be district Tien Giang province)
(project investigate, analyze, write document)
(Bachelor, Economics, GPA 7.21/10, MS Excel, MS Word, Powe

In [10]:
doc_container.to_disk('../data/ner_entities_resumes.spacy')